# Memory in Bedrock AgentCore

Bedrock AgentCore Memory is a managed AWS service that allows you to create and manage memory resources that store conversation contexts for agents.

To get started, you will need to install LlamaIndex and the AgentCore Memory integration

In [ ]:
%pip install llama-index llama-index-memory-bedrock-agentcore llama-index-llms-bedrock-converse

### Bedrock AgentCore Setup Pre-Requisites

1. AWS account with Bedrock AgentCore access
2. Configured AWS credentials (boto3)
3. Created memory resource in AWS Bedrock AgentCore
4. Required IAM permissions:
   1. bedrock-agentcore:CreateEvent
   2. bedrock-agentcore:ListEvents
   3. bedrock-agentcore:RetrieveMemories

## Setup

Create an instance of AgentCoreMemoryContext to setup the memory resources that you will need for building an agent.
   1. Actor id → This is a required field and it is the identifier of the actor (could be an agent or the end-user).
   2. Memory id → This is a required field and it is the identifier of the memory store.
   3. Session id → This is a required field and it is the unique identifier of a particular conversation.
   4. Namespace → This is an optional field and it is used to determine how to extract long term memories. By default it will use “/” as the namespace.
   5. Memory strategy id → This is an optional field and it is the identifier for a memory strategy.

In [ ]:
from llama_index.memory.bedrock_agentcore import (
    AgentCoreMemory,
    AgentCoreMemoryContext,
)

context = AgentCoreMemoryContext(
    actor_id="<REQUIRED>",
    memory_id="<REQUIRED>",
    session_id="<REQUIRED>",
    memory_strategy_id="<OPTIONAL>",
    namespace="<OPTIONAL>",
)
agentcore_memory = AgentCoreMemory(context=context)

## Usage

Initialize the FunctionAgent or ReActAgent with any tool and LLM.

Below we define some dummy tools to have the agent interact with.

In [ ]:
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.tools import FunctionTool


async def call_fn(name: str, message: str):
    """Call the provided name.
    Args:
        name: str (Name of the person, will lookup their phone number)
        message: str (Message to send)
    """
    print(f"Called {name} and left a message.")


async def email_fn(email: str, message: str):
    """Email the provided name.
    Args:
        email: str (Email to send to)
        message: str (Body of the email)
    """
    print(f"Emailing {email} with the provided message.")


llm = BedrockConverse(model="us.anthropic.claude-sonnet-4-20250514-v1:0")


agent = FunctionAgent(
    tools=[email_fn, call_fn],
    llm=llm,
)

Invoke the agent's tool to store this in memory.

In [ ]:
response = await agent.run(
    "Please email johan@coolcompany.com and tell him that his appointement has been moved to 3 PM January 10th, 2025.",
    memory=agentcore_memory,
)

print(str(response))

Query the agent to consult its memory source for the response.

In [ ]:
response = await agent.run(
    "Who have I emailed recently?", memory=agentcore_memory
)

print(str(response))

## References

- [Bedrock AgentCore Memory Documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/memory-getting-started.html)